In [10]:
from utils import simulate_fair_price
from order_book import OrderBook
from trade import Trade

import plotly.express as px
import polars as pl
import numpy as np

## Question 1 : simulating fair prices

In [11]:
# px.line(simulate_fair_price().to_pandas().set_index('Timestamp'))

Question for teacher : what is the economic reality behind these models / what are some good papers explaining these choices
- for a currency, going to 0.01 for a few months and then 50 months later is not realistic / signifies huge macroeconomic events
- what are some papers indicating which parameters make sense in this context

## Question 2 : Order book

In [38]:
# Example Usage
order_book = OrderBook(n_levels=5)
order_book.update_order(price=100.5, size=10, side="bid")
order_book.update_order(price=101.0, size=15, side="bid")
order_book.update_order(price=102.0, size=5, side="ask")
order_book.update_order(price=102.5, size=20, side="ask")
order_book.update_order(price=103.0, size=8, side="ask")

print("Best Bid:", order_book.get_best_bid())  # (101.0, 15)
print("Best Ask:", order_book.get_best_ask())  # (102.0, 5)
print("Order Book:\n", order_book.get_order_book())

Best Bid: (101.0, 15.0, datetime.time(17, 2, 1, 941992))
Best Ask: (102.0, 5.0, datetime.time(17, 2, 1, 943994))
Order Book:
 shape: (3, 6)
┌─────────────────┬──────────┬───────┬───────┬──────────┬─────────────────┐
│ timestamp_bid   ┆ size_bid ┆ bid   ┆ ask   ┆ size_ask ┆ timestamp_ask   │
│ ---             ┆ ---      ┆ ---   ┆ ---   ┆ ---      ┆ ---             │
│ time            ┆ f64      ┆ f64   ┆ f64   ┆ f64      ┆ time            │
╞═════════════════╪══════════╪═══════╪═══════╪══════════╪═════════════════╡
│ 17:02:01.941992 ┆ 15.0     ┆ 101.0 ┆ 102.0 ┆ 5.0      ┆ 17:02:01.943994 │
│ 17:02:01.940997 ┆ 10.0     ┆ 100.5 ┆ 102.5 ┆ 20.0     ┆ 17:02:01.944994 │
│ null            ┆ null     ┆ null  ┆ 103.0 ┆ 8.0      ┆ 17:02:01.945994 │
└─────────────────┴──────────┴───────┴───────┴──────────┴─────────────────┘


In [46]:
order_book.bids.bottom_k(k=1, by='bid').row(index=0) #gives us the worst available bid

(100.5, 10.0, datetime.time(17, 2, 1, 940997))

In [20]:
order_book.delete_order(price=102.0, size=5, side='ask')

## Question 4 : Trade logic

In [4]:
trade = Trade(26, "buy")

In [5]:
order_book.get_order_book()

timestamp_bid,size_bid,bid,ask,size_ask,timestamp_ask
time,f64,f64,f64,f64,time
19:03:32.900706,15.0,101.0,102.0,5.0,19:03:32.902705
19:03:32.897708,10.0,100.5,102.5,20.0,19:03:32.903708
null,null,null,103.0,8.0,19:03:32.904747


In [6]:
ob = trade.update_orderbook_with_trade(orderbook=order_book)

In [17]:
ob.get_order_book()

timestamp_bid,size_bid,bid,ask,size_ask,timestamp_ask
time,f64,f64,f64,f64,time
19:03:32.900706,15.0,101.0,103.0,7.0,19:03:32.958878
19:03:32.897708,10.0,100.5,null,null,null


In [9]:
pl.DataFrame(trade.history)

side,price,size,timestamp
str,f64,f64,datetime[μs]
"""buy""",103.0,1.0,2025-04-14 19:03:32.959876
"""buy""",102.5,1.0,2025-04-14 19:03:32.959876
"""buy""",102.0,1.0,2025-04-14 19:03:32.959876
